##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorBoard에서 Embedding Projector를 사용하여 데이터 시각화하기

<table class="tfo-notebook-buttons" align="left">
  <td>
<img src="https://www.tensorflow.org/images/tf_logo_32px.png"><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tensorboard/tensorboard_projector_plugin.ipynb">TensorFlow.org에서 보기</a>
</td>
  <td>
<img src="https://www.tensorflow.org/images/colab_logo_32px.png"><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tensorboard/tensorboard_projector_plugin.ipynb">Google Colab에서 실행</a>
</td>
  <td>
<img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"><a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/tensorboard_projector_plugin.ipynb">GitHub에서 소스 보기</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tensorboard/tensorboard_projector_plugin.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 개요

**TensorBoard Embedding Projector**를 사용하여 고차원 임베딩을 그래픽으로 나타낼 수 있습니다. 그러면 레이어를 시각화하고 검사 및 이해하는 데 도움이 될 수 있습니다.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=%5C" alt="임베딩 프로젝터의 스크린 샷" width="400">

이 튜토리얼에서는 이러한 유형의 훈련된 레이어를 시각화하는 방법을 알아봅니다.

## 설정

이 튜토리얼에서는 TensorBoard를 사용하여 영화 리뷰 데이터를 분류하기 위해 생성된 임베딩 레이어를 시각화합니다.

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector


## IMDB 데이터

각각에 감정 레이블(긍정/부정)이 있는 25,000개의 IMDB 영화 리뷰 데이터세트를 사용할 것입니다. 각 리뷰는 사전 처리되고 일련의 단어 인덱스(정수)로 인코딩됩니다. 단순화를 위해 단어는 데이터세트의 전체 빈도로 색인화됩니다. 예를 들어 정수 "3"은 모든 리뷰에서 세 번째로 자주 사용되는 단어를 인코딩합니다. 이를 통해 "가장 많이 사용되는 상위 10,000개의 단어만 고려하고 상위 20개의 가장 일반적인 단어는 제거"와 같은 빠른 필터링 작업을 수행할 수 있습니다.

관례적으로, "0"은 특정 단어를 나타내지 않으며, 대신 알 수 없는 단어를 인코딩하는 데 사용됩니다. 튜토리얼 뒷부분에서 "0" 행을 시각화에서 제거합니다.


In [ ]:
(train_data, test_data), info = tfds.load(
    "imdb_reviews/subwords8k",
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True,
    as_supervised=True,
)
encoder = info.features["text"].encoder

# Shuffle and pad the data.
train_batches = train_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
test_batches = test_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
train_batch, train_labels = next(iter(train_batches))


# Keras 임베딩 레이어

[Keras 임베딩 레이어](https://keras.io/layers/embeddings/)는 어휘에 있는 각 단어에 대한 임베딩을 훈련하는 데 사용할 수 있습니다. 각 단어(또는 이 경우 하위 단어)는 모델에서 훈련할 16차원 벡터(또는 임베딩)와 연결됩니다.

단어 임베딩에 대한 자세한 내용은 [이 튜토리얼](https://www.tensorflow.org/tutorials/text/word_embeddings?hl=en)을 참조하세요.

In [9]:
# Create an embedding layer.
embedding_dim = 16
embedding = tf.keras.layers.Embedding(encoder.vocab_size, embedding_dim)
# Configure the embedding layer as part of a keras model.
model = tf.keras.Sequential(
    [
        embedding, # The embedding layer should be the first layer in a model.
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)

# Compile model.
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train model for one epoch.
history = model.fit(
    train_batches, epochs=1, validation_data=test_batches, validation_steps=20
)

2500/2500 [==============================] - 13s 5ms/step - loss: 0.5330 - accuracy: 0.6769 - val_loss: 0.4043 - val_accuracy: 0.7800


## TensorBoard용 데이터 저장하기

TensorBoard는 tensorflow 프로젝트의 로그에서 텐서와 메타데이터를 읽습니다. 아래에서 로그 디렉터리의 경로는 `log_dir`로 지정됩니다. 이 튜토리얼에서는 `/logs/imdb-example/`을 사용합니다.

데이터를 Tensorboard에 로드하려면 모델의 특정 관심 레이어를 시각화할 수 있는 메타데이터와 함께 훈련 검사점을 해당 디렉터리에 저장해야 합니다. 

In [ ]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='/logs/imdb-example/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for subwords in encoder.subwords:
    f.write("{}\n".format(subwords))
  # Fill in the rest of the labels with "unknown".
  for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    f.write("unknown #{}\n".format(unknown))


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(model.layers[0].get_weights()[0][1:])
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
# Now run tensorboard against on log data we just saved.
%tensorboard --logdir /logs/imdb-example/

<!-- <img class="tfo-display-only-on-site" src="images/embedding_projector.png?raw=1"/> -->

## 분석

TensorBoard Projector는 임베딩을 해석하고 시각화하기 위한 훌륭한 도구입니다. 대시보드를 통해 사용자는 특정 용어를 검색하고 임베딩(낮은 차원) 공간에서 서로 인접한 단어를 강조 표시할 수 있습니다. 이 예에서는 Wes **Anderson**과 Alfred **Hitchcock**이 둘 다 다소 중립적인 용어이지만 다른 맥락에서 참조된다는 것을 알 수 있습니다.

<!-- <img class="tfo-display-only-on-site" src="images/embedding_projector_hitchcock.png?raw=1"/> -->

이 공간에서 Hitchcock은 `nightmare`와 같은 단어에 더 가깝습니다. 이는 그가 '서스펜스의 귀재'로 알려져 있기 때문일 것입니다. 반면 Anderson은 그의 끊임없는 디테일과 훈훈한 스타일에 어울리는 `heart`에 더 가깝습니다.

<!-- <img class="tfo-display-only-on-site" src="images/embedding_projector_anderson.png?raw=1"/> -->